In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
% matplotlib notebook
pd.set_option('display.max_columns', 100)

import folium
from folium.plugins import HeatMap

from textblob import TextBlob
from textblob import Word
from textblob_de import TextBlobDE
from textblob_de import Word as WordDE
from nltk.corpus import stopwords
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

from currency_converter import CurrencyConverter
import os

<h2> 0. Load Data and create dataframe for further analysis</h2>

In [ ]:
listing = pd.read_csv('listings.csv', encoding = 'utf-8')
calendar = pd.read_csv('calendar.csv')
reviews = pd.read_csv('reviews.csv')

In [ ]:
listing_df = listing[['id','description', 'host_id', 'host_since', 'host_response_time','host_response_rate', 'host_is_superhost',
     'host_verifications','host_identity_verified','neighbourhood_cleansed','latitude', 'longitude',
       'is_location_exact','property_type', 'room_type','square_feet', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
       'price', 'weekly_price', 'monthly_price','security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights','availability_365','number_of_reviews',
       'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value','instant_bookable',
       'is_business_travel_ready','cancellation_policy','require_guest_profile_picture', 'require_guest_phone_verification']]

listing_df['price'] = listing_df['price'].str.replace('$','')
listing_df['price'] = listing_df['price'].str.replace(',','')
listing_df['price'] = pd.to_numeric(listing_df['price'])
listing_df['price'] = listing_df['price'].fillna(listing_df['price'].mean())

listing_df['extra_people'] = listing_df['extra_people'].str.replace('$','')
listing_df['extra_people'] = listing_df['extra_people'].str.replace(',','')
listing_df['extra_people'] = pd.to_numeric(listing_df['price'])
listing_df['extra_people'] = listing_df['extra_people'].fillna(listing_df['extra_people'].mean())

cc = CurrencyConverter()
listing_df['extra_people'] = listing_df['extra_people'].apply(lambda x: cc.convert(x,'USD','EUR'))
listing_df['extra_people'] = listing_df['extra_people'].apply(lambda x: cc.convert(x,'USD','EUR'))

# clean data from inactive listings (inactive if no review in last two years and not bookable over next year)
listing_df['last_review'] = pd.to_datetime(listing_df['last_review'])
listing_df['reviewed_last2years'] = listing_df['last_review'] > dt.datetime(year=2016,month=11,day=13)
listing_df['bookable_nextyear'] = listing_df['id'].isin(calendar[calendar['available'] == 't']['listing_id'])
inactive_listings = listing_df[(listing_df['bookable_nextyear'] == False) & (listing_df['reviewed_last2years'] == False)]
active_listings = listing_df[~listing_df['id'].isin(inactive_listings['id'])]
listing_df = active_listings

<h1> 1. Information about listings and hosts </h1>

In [ ]:
listing_df['host_since'] = pd.to_datetime(listing_df['host_since'])
listing_df['first_review'] = pd.to_datetime(listing_df['first_review'])

<h3> Number of listings and hosts</h3>

In [ ]:
number_of_listings = pd.DataFrame()
number_of_listings['host_id']= listing_df['host_id'].value_counts().index
number_of_listings['listings']= listing_df['host_id'].value_counts().values

In [ ]:
nol_dist = {"1": len(number_of_listings[number_of_listings['listings']== 1]),
            "2-5": len(number_of_listings[number_of_listings['listings'] > 1]) - len(number_of_listings[number_of_listings['listings'] > 5]),
            "6-20": len(number_of_listings[number_of_listings['listings'] > 5]) - len(number_of_listings[number_of_listings['listings'] > 20]),
            "20+": len(number_of_listings[number_of_listings['listings'] > 20])}

plt.cla()

diff = nol_dist.keys
y_pos = np.arange(len(nol_dist))
values = list(nol_dist.values())

fig = plt.figure().add_subplot(111)
plt.yticks(y_pos, list(nol_dist.keys()))
plt.subplots_adjust(left=0.15)
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.yticks(fontsize=9, color='darkgrey')
plt.gca().invert_yaxis()

#German version 
#plt.suptitle('Wer steckt hinter den Gastgebern: Privatpersonen oder Unternehmen?',fontsize=12)
#plt.title('Anzahl der Gastgeber mit einem oder mehreren Objekten', fontsize=10, color='grey', fontweight='bold')
#plt.ylabel("Anzahl Objekte")
#
#for i in range(len(nol_dist)):
#    fig.annotate('%s Gastgeber' %values[i], xy=(40,0.05 + i), fontsize='medium', color='black')
#
#plt.barh(y_pos,values)
#plt.savefig('German Graphs/anzahl_mietobjekte_pro_gastgeber.png')

#English version 
plt.suptitle('Who are the Airbnb hosts: private persons or companies?',fontsize=12)
plt.title('Number of hosts with one or more listings', fontsize=10, color='grey', fontweight='bold')
plt.ylabel("Number of listings")

for i in range(len(nol_dist)):
    fig.annotate('%s Hosts' %values[i], xy=(40,0.05 + i), fontsize='medium', color='black')

plt.barh(y_pos,values)
plt.savefig('English Graphs/number_of_listings_per_host.png')


<h3> Development of number of hosts and listings </h3>
assumptions: we use the date of the first review as an indicator when a listing was added to Airbnb

In [ ]:
hosts_per_year= {
    "2008": len(listing_df[listing_df['host_since'] <= dt.date(year=2008, month=12, day=31)]['host_id'].unique()),
    "2009": len(listing_df[listing_df['host_since'] <= dt.date(year=2009, month=12, day=31)]['host_id'].unique()) ,
    "2010": len(listing_df[listing_df['host_since'] <= dt.date(year=2010, month=12, day=31)]['host_id'].unique()) ,
    "2011": len(listing_df[listing_df['host_since'] <= dt.date(year=2011, month=12, day=31)]['host_id'].unique()) ,
    "2012": len(listing_df[listing_df['host_since'] <= dt.date(year=2012, month=12, day=31)]['host_id'].unique()) ,
    "2013": len(listing_df[listing_df['host_since'] <= dt.date(year=2013, month=12, day=31)]['host_id'].unique()) ,
    "2014": len(listing_df[listing_df['host_since'] <= dt.date(year=2014, month=12, day=31)]['host_id'].unique()) ,
    "2015": len(listing_df[listing_df['host_since'] <= dt.date(year=2015, month=12, day=31)]['host_id'].unique()) ,
    "2016": len(listing_df[listing_df['host_since'] <= dt.date(year=2016, month=12, day=31)]['host_id'].unique()) ,
    "2017": len(listing_df[listing_df['host_since'] <= dt.date(year=2017, month=12, day=31)]['host_id'].unique()) ,
    "2018": len(listing_df[listing_df['host_since'] <= dt.date(year=2018, month=12, day=31)]['host_id'].unique())
}

fig = plt.figure().add_subplot(111)
x = hosts_per_year.keys()
y= hosts_per_year.values()

plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=20)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold')

#German version
#plt.ylabel('Anzahl an aktiven Gastgebern')
#plt.suptitle('Wie stark ist die Anbieterseite gewachsen?',fontsize=12)
#plt.title('Entwicklung der Anzahl an Gastgeber seit 2008', fontsize=10, color='grey', fontweight='bold')
#
#for i,j in zip(x,y):
#    fig.annotate('%s' %j, xy=(i,j), xytext=(10,0), textcoords='offset points',)
#    
#plt.plot(x,y, linestyle='--', marker='o', color='lightskyblue')
#plt.savefig('German Graphs/entwicklung_anzahl_gastgeber.png')

#English version
plt.ylabel('Number of active hosts')
plt.suptitle('How strong was the growth on the supply side?',fontsize=12)
plt.title('Development of number of hosts since 2008', fontsize=10, color='grey', fontweight='bold')

for i,j in zip(x,y):
    fig.annotate('%s' %j, xy=(i,j), xytext=(10,0), textcoords='offset points',)
    
plt.plot(x,y, linestyle='--', marker='o', color='lightskyblue')
plt.savefig('English Graphs/development_number_of_hosts.png')


In [ ]:
listings_per_year= {
    "2008": len(listing_df[listing_df['first_review'] <= dt.date(year=2008, month=12, day=31)]),
    "2009": len(listing_df[listing_df['first_review'] <= dt.date(year=2009, month=12, day=31)]) ,
    "2010": len(listing_df[listing_df['first_review'] <= dt.date(year=2010, month=12, day=31)]) ,
    "2011": len(listing_df[listing_df['first_review'] <= dt.date(year=2011, month=12, day=31)]) ,
    "2012": len(listing_df[listing_df['first_review'] <= dt.date(year=2012, month=12, day=31)]) ,
    "2013": len(listing_df[listing_df['first_review'] <= dt.date(year=2013, month=12, day=31)]) ,
    "2014": len(listing_df[listing_df['first_review'] <= dt.date(year=2014, month=12, day=31)]) ,
    "2015": len(listing_df[listing_df['first_review'] <= dt.date(year=2015, month=12, day=31)]) ,
    "2016": len(listing_df[listing_df['first_review'] <= dt.date(year=2016, month=12, day=31)]) ,
    "2017": len(listing_df[listing_df['first_review'] <= dt.date(year=2017, month=12, day=31)]) ,
    "2018": len(listing_df[listing_df['first_review'] <= dt.date(year=2018, month=12, day=31)])
}

fig = plt.figure().add_subplot(111)
x = listings_per_year.keys()
y= listings_per_year.values()
plt.xlabel("Jahr")
plt.ylabel("Anzahl der Reviews")

for i,j in zip(x,y):
    fig.annotate('%s' %j, xy=(i,j), xytext=(10,0), textcoords='offset points',)

plt.plot(x,y, linestyle='--', marker='o', color='lightskyblue')

<h3> Room types </h3>

In [ ]:
room_types = listing_df['room_type'].value_counts()
plt.cla()

plt.axis('equal')
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=20)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold')

#German version
#plt.suptitle('Was bieten die Gastgeber ihren Gästen an?',fontsize=12)
#plt.title('Verteilung der Objekttypen', fontsize=10, color='grey', fontweight='bold')
#labels=['Gesamte Wohnung','Privatzimmer','Geteiltes Zimmer']
#colors = ['dodgerblue','deepskyblue','powderblue']
#patches, texts, _ = plt.pie(room_types.values,autopct='%1.1f%%', colors=colors, startangle=90)
#plt.legend(patches, labels, loc="best")
#plt.legend(labels)
#
#plt.savefig('German Graphs/mietobjekt_typen.png')

#English version
plt.suptitle('What do hosts offer their guests?',fontsize=12)
plt.title('Percentages of different room types', fontsize=10, color='grey', fontweight='bold')
labels=['Whole apartment','Private room','Shared Room']
colors = ['dodgerblue','deepskyblue','powderblue']
patches, texts, _ = plt.pie(room_types.values,autopct='%1.1f%%', colors=colors, startangle=90)
plt.legend(patches, labels, loc="best")
plt.legend(labels)

plt.savefig('English Graphs/room_types.png')

<h3> Additional information about listings and hosts </h3>

In [ ]:
print("Date of first listing published: " + str((listing_df['host_since'].min())))
print("Number of listings:" + str(len(listing_df['id'].unique())))
print("Number of hosts:" + str(len(listing_df['host_id'].unique())))
print("Number of Superhosts:" + str(len(listing_df[listing_df['host_is_superhost']== 't'])))
print("Biggest Host:" + str(number_of_listings[number_of_listings['listings'] == number_of_listings['listings'].max()]['host_id'].values[0]))
print("Number of listings biggest host:" + str(number_of_listings.max()['listings']))
print("Number of hosts with just one listing: " + str(len(number_of_listings[number_of_listings['listings'] == 1] )))

<h1>2. Location </h1>

In [ ]:
neighbourhoods = pd.DataFrame()
neighbourhoods['name'] = listing_df['neighbourhood_cleansed'].value_counts().index
neighbourhoods['count'] = listing_df['neighbourhood_cleansed'].value_counts().values
neighbourhoods['longitude'] = np.arange(len(neighbourhoods['name']))
neighbourhoods['latitude'] = np.arange(len(neighbourhoods['name']))

for name in neighbourhoods['name']:
    neighbourhoods.at[neighbourhoods['name'] == name,'longitude'] = listing_df[listing_df['neighbourhood_cleansed'] == name]['longitude'].mean()
    neighbourhoods.at[neighbourhoods['name'] == name,'latitude'] = listing_df[listing_df['neighbourhood_cleansed'] == name]['latitude'].mean()
  

neighbourhoods['name'] = neighbourhoods['name'].str.replace('Landstra§e' ,'Landstraße')
neighbourhoods['name'] = neighbourhoods['name'].str.replace('Fnfhaus', 'Fünfhaus')
neighbourhoods['name'] = neighbourhoods['name'].str.replace('Whring', 'Währing')
neighbourhoods['name'] = neighbourhoods['name'].str.replace('Dbling', 'Döbling')

In [ ]:
plt.cla()

names = neighbourhoods.index
y_pos = np.arange(len(names))
values = neighbourhoods.values

fig = plt.figure().add_subplot(111)
plt.yticks(y_pos, names)
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.yticks(fontsize=9, color='darkgrey', rotation= 20) 
plt.subplots_adjust(left=0.25)
plt.gca().invert_yaxis()

for i in range(len(names)):
    fig.annotate('%s' %values[i], xy=(1,0.3 + i), fontsize='small', color='white')

#German version
#plt.suptitle('In welchen Bezirken sind die meisten Airbnb Wohnungen?',fontsize=12)
#plt.title('Anzahl Mietobjekte nach Wiener Bezirk', fontsize=10, color='grey', fontweight='bold')
#
#plt.barh(y_pos,values, height = 0.9)
#plt.savefig('German Graphs/anzahl_listings_nach_bezirken.png')

#English version
plt.suptitle('In which districts are the most listings?',fontsize=12)
plt.title('Number of listings in each district', fontsize=10, color='grey', fontweight='bold')

plt.barh(y_pos,values, height = 0.9)
plt.savefig('English graphs/number_of_listings_after_districts.png')



<h2> Map visualization </h2>

In [ ]:
#Generate map of Vienna
base_map = folium.Map(location=[48.209,16.37], zoom_start=11.5)
#Generate HeatMap layer on base_map

for i in range(0,11):
    temp = listing_df[listing_df['first_review'] <= dt.date(year=2008+i, month=12, day=31)]
    temp['count'] = 1
    HeatMap(data=temp[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
    base_map.save('heatmap200' + str(8+i) + '.html')


In [ ]:
#Generate Marker for every district
i=0;
for name in neighbourhoods['name']:
    latitude = neighbourhoods[neighbourhoods['name'] == name]['latitude'][i]
    longitude = neighbourhoods[neighbourhoods['name'] == name]['longitude'][i]
    count = neighbourhoods[neighbourhoods['name'] == name]['count'][i]
    #load prices_and_neighbourhood first
    price = prices_and_neighbourhood[prices_and_neighbourhood['neighbourhood']==name]['mean_price'].mean()
    popup = "<h3>" + name + "</h3> <p><strong>Anzahl Apartments: </strong>" + str(count) + "</p> <p><strong>Durchschnittlicher Preis: </strong>" + str(round(price)) + "€</p>"
    folium.CircleMarker([latitude, longitude],radius=10,fill=True, popup=popup ,tooltip=name).add_to(base_map)
    i += 1

<h1>3. Reviews </h1>

In [ ]:
comments = pd.read_csv('comments_preprocessed.csv')
comments = comments.dropna()
comments = comments[comments['language'] != 'error']

<h4> Further preprocessing </h4>

In [ ]:
#remove rare and stopwords
en_comments = comments[comments['language'] == 'en']
de_comments = comments[comments['language'] == 'de']
de_stop = stopwords.words('german')
en_stop = stopwords.words('english')
de_comments['comments'] = de_comments['comments'].apply(lambda x: " ".join(x for x in x.split() if x not in de_stop))
en_comments['comments'] = en_comments['comments'].apply(lambda x: " ".join(x for x in x.split() if x not in en_stop))
en_rare = pd.Series(' '.join(en_comments['comments']).split()).value_counts()[-200:]
de_rare = pd.Series(' '.join(de_comments['comments']).split()).value_counts()[-200:]
en_rare = list(en_rare.index)
de_rare = list(de_rare.index)
de_comments['comments'] = de_comments['comments'].apply(lambda x: " ".join(x for x in x.split() if x not in de_rare))
en_comments['comments'] = en_comments['comments'].apply(lambda x: " ".join(x for x in x.split() if x not in en_rare))

In [ ]:
#lemmatize english
en_comments['comments'] = en_comments['comments'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#lemmatize german - not implemented
#de_comments['comments'] = de_comments['comments'].apply(lambda x: " ".join([WordDE(word).lemmatize() for word in x.split()]))

<h3> Frequent positive feedback </h3>

In [ ]:
en_comments['sentiment_polarity'] = en_comments['comments'].apply(lambda x: TextBlob(x).sentiment.polarity)
pos_en_comments = en_comments[en_comments['sentiment_polarity'] > 0]
pos_freq = en_rare = pd.Series(' '.join(pos_en_comments['comments']).split()).value_counts()[:30]
pos_freq

<h3> General infos about reviews </h3>

In [ ]:
total_reviews = len(reviews['id'])
listings_with_review = len(reviews['listing_id'].unique())
reviews_per_listing = reviews['listing_id'].value_counts()
print("Reviews insgesamt: " + str(total_reviews))
print("Listings mit Reviews : " + str(listings_with_review) + ' (=' + str(round((listings_with_review / total_listings) *100 , 2)) + '%)')

<h3> Language of reviews </h3>

In [ ]:
languages_of_listings = comments['language'].value_counts() / len(comments['language'])
print(languages_of_listings['en'])
print(languages_of_listings['de'])
print(languages_of_listings['fr'])

<h1>4. Pricing </h1>

In [ ]:
calendar['price'] = calendar['price'].str.replace('$','')
calendar['price'] = calendar['price'].str.replace(',','')
calendar['price'] = pd.to_numeric(calendar['price'])
calendar['price'] = calendar.groupby('listing_id').transform(lambda x: x.fillna(x.mean()))
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['weekday'] = calendar['date'].apply(lambda x: x.weekday())

In [ ]:
cc = CurrencyConverter()
calendar['price'] = calendar['price'].apply(lambda x: cc.convert(x,'USD','EUR'))

In [ ]:
print("Average Price: " + str(round(calendar['price'].mean(), 2)))
print("Most expensive price: " + str(round(calendar['price'].max(), 2)))
print("Cheapest price: " + str(round(calendar['price'].min(), 2)))
print("Average price on new years eve: " + str(round(calendar[calendar['date'] == dt.datetime(year=2018, month=12, day=31)]['price'].mean(), 2)))

<h3> Development of average prives over the year </h3>

In [ ]:
dates_with_mean_prices = pd.DataFrame()
dates_with_mean_prices['date'] = calendar.groupby(['date'])['price'].mean().index
dates_with_mean_prices['price'] = calendar.groupby(['date'])['price'].mean().values


dates_with_mean_prices['week'] = pd.Series()
week = 0
for i in range(len(dates_with_mean_prices['week'])):
    if(i%7 == 0):
        week = week + 1
    dates_with_mean_prices.at[i,'week'] = week

dates_with_mean_prices = dates_with_mean_prices[dates_with_mean_prices['week'] != 53]

In [ ]:
plt.cla()

x = dates_with_mean_prices.groupby('week').sum()['price'].index
y= dates_with_mean_prices.groupby('week').sum()['price'].values / 7

fig,ax = plt.subplots()

ax.set_xticks([0.8,5,9,13.5,18,22.5,27,31.5,36,40.5,45,49.5])
plt.subplots_adjust(bottom=0.25)

plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=90)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold') 
plt.plot(x,y, color='lightskyblue')

#German Version
#labels = ['November', 'Dezember','Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober']
#plt.suptitle('Saisonalität der Preise im Jahresverlauf',fontsize=12)
#plt.title('Entwicklung des durchnittlichen Preis pro Nacht', fontsize=10, color='grey', fontweight='bold')
#plt.ylabel('Preis pro Nacht in €')
#ax.set_xticklabels(labels, minor=False)
#plt.savefig('German Graphs/saisonalität_preise_jahr.png')

#English Version
labels = ['November', 'December','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October']
plt.suptitle('Saisonality of prices over the year',fontsize=12)
plt.title('Development of average price per night', fontsize=10, color='grey', fontweight='bold')
plt.ylabel('Price per night in €')
ax.set_xticklabels(labels, minor=False)
plt.savefig('English Graphs/saiosonality_prices_year.png')

<h3> Average prices on different weekdays </h3>

In [ ]:
days_with_mean_prices = calendar.groupby(['weekday'])['price'].mean()

plt.cla()

fig,ax = plt.subplots()

y_pos = np.arange(len(days_with_mean_prices.index))
values= days_with_mean_prices.values
ax.set_xticks(y_pos)   

plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=20)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold') 
plt.plot(y_pos,values, linestyle='--', marker='o', color='lightskyblue')

#German version 
#labels= ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag','Sonntag']
#ax.set_xticklabels(labels, minor=False, rotation=10)
#plt.suptitle('Änderungen der Preise im Wochenverlauf',fontsize=12)
#plt.title('Entwicklung des durchnittlichen Preis pro Nacht', fontsize=10, color='grey', fontweight='bold')
#plt.ylabel('Preis pro Nacht in €') 
#plt.savefig('German Graphs/änderungen_preise_woche.png')

#English version
labels= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
ax.set_xticklabels(labels, minor=False, rotation=10)
plt.suptitle('Price changes over the week',fontsize=12)
plt.title('Development of average price per night', fontsize=10, color='grey', fontweight='bold')
plt.ylabel('Price per night in €') 
plt.savefig('English Graphs/changes_prices_week.png')


<h3> Average prices of neighbourhoods </h3>

In [ ]:
prices_and_neighbourhood = pd.DataFrame()
prices_and_neighbourhood['listing_id'] = calendar.groupby(['listing_id'])['price'].mean().index
prices_and_neighbourhood['mean_price'] = calendar.groupby(['listing_id'])['price'].mean().values
listing_df.sort_values(by='id', inplace=True, )
prices_and_neighbourhood.sort_values(by='listing_id', inplace=True, )
for i in range(len(listing_df['id'])):
    if listing_df.iloc[i]['id'] == prices_and_neighbourhood.iloc[i]['listing_id']:
        continue
    else: 
        print(listing_df.iloc[i]['id'])
        break
        
prices_and_neighbourhood['neighbourhood'] = listing_df[listing_df.id != 116884]['neighbourhood_cleansed']
prices_and_neighbourhood['neighbourhood']= prices_and_neighbourhood['neighbourhood'].str.replace('Landstra§e' ,'Landstraße')
prices_and_neighbourhood['neighbourhood']= prices_and_neighbourhood['neighbourhood'].str.replace('Fnfhaus', 'Fünfhaus')
prices_and_neighbourhood['neighbourhood']= prices_and_neighbourhood['neighbourhood'].str.replace('Whring', 'Währing')
prices_and_neighbourhood['neighbourhood']= prices_and_neighbourhood['neighbourhood'].str.replace('Dbling', 'Döbling')

plt.cla()

names = prices_and_neighbourhood.groupby(['neighbourhood'])['mean_price'].mean().sort_values(ascending = False).index
y_pos = np.arange(len(names))
values = prices_and_neighbourhood.groupby(['neighbourhood'])['mean_price'].mean().sort_values(ascending = False).values


fig = plt.figure().add_subplot(111)
plt.yticks(y_pos, names)
plt.subplots_adjust(left=0.25)
plt.gca().invert_yaxis()


plt.xticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.yticks(fontsize=9, color='darkgrey', rotation= 20) 

for i in range(len(names)):
    fig.annotate('%s€' %round(values[i], 2), xy=(1,0.3 + i), fontsize='small', color='white')

#German version
#plt.suptitle('Wo sind Airbnb Wohnungen am teuersten?',fontsize=12)
#plt.title('Durschnitllicher Preis pro Nacht nach Bezirk', fontsize=10, color='grey', fontweight='bold')
#plt.barh(y_pos,values)
#plt.savefig('German Graphs/preise_nach_bezirk.png')

#English version 
plt.suptitle('Which district is the most expensive area?',fontsize=12)
plt.title('Average price per night of each district', fontsize=10, color='grey', fontweight='bold')
plt.barh(y_pos,values)
plt.savefig('English Graphs/prices_for_districts.png')

In [ ]:
prices_and_neighbourhood

<h3> Distribution of and Average Prices for different apartment sizes </h3>

In [ ]:
one_bedroom = listing_df[listing_df['bedrooms'] == 1]
two_bedrooms = listing_df[listing_df['bedrooms'] == 2]
three_bedrooms = listing_df[listing_df['bedrooms'] == 3]
four_or_more_bedrooms = listing_df[listing_df['bedrooms'] >= 4]

In [ ]:
dist_apartment_sizes = {"1": len(one_bedroom), "2":len(two_bedrooms), "3":len(three_bedrooms), "4+": len(four_or_more_bedrooms)}

plt.cla()

values = dist_apartment_sizes.values()

plt.axis('equal')
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=20)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold')

colors = ['dodgerblue','deepskyblue','lightskyblue', "lightblue"]

#German version
#plt.suptitle('Wie groß sind Airbnb Wohnungen?',fontsize=12)
#plt.title('Anzahl der Schlafzimmer', fontsize=10, color='grey', fontweight='bold')
#labels=['1 Schlafzimmer','2 Schlafzimmer','3 Schlafzimmer', "4+ Schlafzimmer"]
#
#patches, texts, _ = plt.pie(values,autopct='%1.1f%%', colors=colors, startangle=90, pctdistance=1.15,   labeldistance=1.2)
#plt.legend(patches, labels, loc="best")
#plt.legend(labels)
#
#plt.savefig('German Graphs/größe_der_Wohnungen.png')

#English version
plt.suptitle('How big are Airbnb apartments?',fontsize=12)
plt.title('Number of bedrooms', fontsize=10, color='grey', fontweight='bold')
labels=['1 bedroom ','2 bedrooms','3 bedrooms', "4+ bedrooms"]

patches, texts, _ = plt.pie(values,autopct='%1.1f%%', colors=colors, startangle=90, pctdistance=1.15,   labeldistance=1.2)
plt.legend(patches, labels, loc="best")
plt.legend(labels)

plt.savefig('English Graphs/apartment_sizes.png')

In [ ]:
avg_price_apartment_sizes = {"1": one_bedroom['price'].mean(),
                             "2": two_bedrooms['price'].mean(),
                             "3": three_bedrooms['price'].mean(),
                             "4+": four_or_more_bedrooms['price'].mean()}
plt.cla()

names = avg_price_apartment_sizes.keys()
y_pos = np.arange(len(names))
values = list(avg_price_apartment_sizes.values())

fig = plt.figure().add_subplot(111)
plt.yticks(y_pos, names)
plt.subplots_adjust(left=0.1)
plt.gca().invert_yaxis()

plt.xticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.yticks(fontsize=9, color='darkgrey')

i=0
for key in names:
    fig.annotate('%s€' %round(avg_price_apartment_sizes[key], 2), xy=(1,i), fontsize='medium', color='white')
    i += 1

#German version
#plt.suptitle('Wie hoch ist der Preis pro Nacht?',fontsize=12)
#plt.title('Durschnittlicher Preis pro Nacht nach Größe der Wohnung', fontsize=10, color='grey', fontweight='bold')
#plt.xlabel('Preis pro Nacht in €')
#plt.ylabel('Anzahl der Schlafzimmer')
#plt.barh(y_pos,values)

#plt.savefig('German Graphs/preise_nach_größe.png')

#English version
plt.suptitle('How high is the price per night?',fontsize=12)
plt.title('Average price per night for different apartment sizes', fontsize=10, color='grey', fontweight='bold')
plt.xlabel('Price per night in €')
plt.ylabel('Number of bedrooms')
plt.barh(y_pos,values)

plt.savefig('English Graphs/prices_by_apartment_size.png')

<h3> Sample calculation for monthly income </h3>
- occupancy rate of 90% 
- district: leopoldstadt
- number of bedrooms: 1
- average people per visit: 1.5

In [ ]:
one_bedroom_in_leopoldstadt = one_bedroom[one_bedroom['neighbourhood_cleansed'] == 'Leopoldstadt']
price_total = (one_bedroom_in_leopoldstadt['price'].sum()) + 0.5* (one_bedroom_in_leopoldstadt[one_bedroom_in_leopoldstadt['guests_included'] == 1]['extra_people'].sum())
avg_price = price_total / len(one_bedroom_in_leopoldstadt)
monthly_income = (avg_price * 365 * 0.85)/12

<h1>5. Demand </h1>
<h3> Occupancy rate </h3>
<h4>Assumptions:</h4>
- Review Rate of 50% is used to convert reviews to estimated bookings
- the average length is assumed to be 3 nights
- the average length of stay multiplied by the estimated bookings for each listing over a period gives the occupancy rate
- If a listing has a higher minimum nights value than the average length of stay, the minimum nights value was used instead.The opposite applies to maximum nights.
- occupancy rate was capped at 99%

In [ ]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews_per_listing = reviews[reviews['date'] >= dt.date(year=2017, month=11, day=15)]['listing_id'].value_counts().sort_index()

In [ ]:
occupancy_df = pd.DataFrame()
occupancy_df['listing_id'] = reviews_per_listing.index
occupancy_df['reviews_count'] = reviews_per_listing.values
occupancy_df['bookings'] = reviews_per_listing.values*2

In [ ]:
df = listing_df[listing_df.id.isin(occupancy_df.listing_id)]
df.sort_values(by='id')
df = df.reset_index(drop=True)
occupancy_df['price'] = df['price']
occupancy_df['minimum_nights'] = df['minimum_nights']
occupancy_df['maximum_nights'] = df['maximum_nights']

In [ ]:
occupancy_df['average_length'] = np.full(shape=len(occupancy_df['listing_id']), fill_value=3)
for i in range(len(occupancy_df['average_length'])):
    if occupancy_df.iloc[i]['average_length'] < occupancy_df.iloc[i]['minimum_nights']:
        occupancy_df.at[i, 'average_length'] = occupancy_df.iloc[i]['minimum_nights']
        continue
    if occupancy_df.iloc[i]['average_length'] > occupancy_df.iloc[i]['maximum_nights']:
        occupancy_df.at[i, 'average_length'] = occupancy_df.iloc[i]['maximum_nights']

In [ ]:
occupancy_df['occupancy_rate'] = (occupancy_df['average_length']*occupancy_df['bookings'])/365
for i in range(len(occupancy_df['occupancy_rate'])):
    if occupancy_df.iloc[i]['occupancy_rate'] >= 0.99: 
        occupancy_df.at[i,'occupancy_rate'] = 0.99

In [ ]:
dist_or = { '<20%':len(occupancy_df[occupancy_df['occupancy_rate'] < 0.20]),
            '21-75%':len(occupancy_df[occupancy_df['occupancy_rate'] > 0.20]) - len(occupancy_df[occupancy_df['occupancy_rate'] > 0.75]), 
            '75%+':len(occupancy_df[occupancy_df['occupancy_rate'] > 0.75])}

plt.cla()
y_pos = np.arange(len(dist_or))
values = list(dist_or.values())
fig = plt.figure().add_subplot(111)
plt.yticks(y_pos, list(dist_or.keys()))
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.yticks(fontsize=9, color='darkgrey',fontweight='bold') 

plt.subplots_adjust(left=0.15)
plt.gca().invert_yaxis()

#German version
#plt.ylabel('Auslastung in %')
#plt.suptitle('Wie hoch ist die Auslastung bei Airbnb Vermietung?',fontsize=14)
#plt.title('Aufteilung der Mietobjekte in drei Kategorien', fontsize=10, color='grey', fontweight='bold')
#for i in range(len(dist_or)):
#    fig.annotate('%s Objekte' %values[i], xy=(40,0.01 + i), fontsize='small', color='white')
#
#plt.barh(y_pos,values)
#plt.savefig('German graphs/auslastung_allgemein.png')

#English version
plt.ylabel('Occupancy in %')
plt.suptitle('How high is the occupancy rate of Airbnb rentals?',fontsize=14)
plt.title('Three categories', fontsize=10, color='grey', fontweight='bold')
for i in range(len(dist_or)):
    fig.annotate('%s Listings' %values[i], xy=(40,0.01 + i), fontsize='small', color='white')

plt.barh(y_pos,values)
plt.savefig('English Graphs/general_occupancy.png')

<h3> Characteristics of listings with a high/low occupancy rate </h3>

In [ ]:
hior_listings = occupancy_df[occupancy_df['occupancy_rate'] < 0.90]['listing_id'].reset_index(drop=True)
hior_df = listing_df[listing_df.id.isin(hior_listings)].reset_index(drop=True)
hior_df['host_response_rate'] = hior_df['host_response_rate'].str.replace('%','')
hior_df['host_response_rate'] = pd.to_numeric(hior_df['host_response_rate'])
hior_df['security_deposit'] = hior_df['security_deposit'].str.replace('$','')
hior_df['security_deposit'] = hior_df['security_deposit'].str.replace(',','')
hior_df['security_deposit'] = pd.to_numeric(hior_df['security_deposit'])
hior_df['cleaning_fee'] = hior_df['cleaning_fee'].str.replace('$','')
hior_df['cleaning_fee'] = pd.to_numeric(hior_df['cleaning_fee'])

cc = CurrencyConverter()
hior_df['security_deposit'] = hior_df['security_deposit'].apply(lambda x: cc.convert(x,'USD','EUR'))
hior_df['cleaning_fee'] = hior_df['cleaning_fee'].apply(lambda x: cc.convert(x,'USD','EUR'))
hior_df['price'] = hior_df['price'].apply(lambda x: cc.convert(x,'USD','EUR'))


wifi = 0
no_wifi = 0
tv = 0
no_tv = 0
for i in range(len(hior_df['amenities'])):
    if 'Wifi' in hior_df['amenities'][i]:
        wifi = wifi + 1
    else: no_wifi = no_wifi + 1
    if 'TV' in hior_df['amenities'][i]:
        tv = tv + 1
    else: no_tv = no_tv + 1

description_lang = pd.DataFrame()
for i in range(0,len(hior_df['description'])):
    try: 
        description_lang= description_lang.append({'lang': detect(hior_df.iloc[i]['description'])}, ignore_index=True)
        #print(description_lang.iloc[i]['lang'])
        if(i%100 == 0):
            print(str(len(hior_df['description']) - i) + " remaining")
    except: 
        description_lang= description_lang.append({'lang': 'error'}, ignore_index=True)
        #print(description_lang.iloc[i]['lang'])
        
hior_df['description_lang'] = description_lang
hior_df.head()

In [ ]:
dist_total = (listing_df['neighbourhood_cleansed'].value_counts() / len(listing_df)) * 100
dist_hior = (hior_df['neighbourhood_cleansed'].value_counts() / len(hior_df)) * 100
diff = dist_total - dist_hior

In [ ]:
perc_host_answer_onehour= round(hior_df['host_response_time'].value_counts()['within an hour']/len(hior_df),4)*100
answer_rate = round(hior_df['host_response_rate'].mean(), 2)

perc_superhost =round(round(hior_df['host_is_superhost'].value_counts()['t']/len(hior_df),4)*100,2)

review_score_cleanliness = round(hior_df['review_scores_cleanliness'].mean(),2)
review_score_checkin = round(hior_df['review_scores_checkin'].mean(),2)
review_score_communication = round(hior_df['review_scores_communication'].mean(),2)
review_score_location = round(hior_df['review_scores_location'].mean(),2)
review_score_value = round(hior_df['review_scores_value'].mean(),2)

perc_instant_bookable = round(hior_df['instant_bookable'].value_counts()['t'] / len(hior_df), 4) * 100

perc_identiy_verified = round(hior_df['host_identity_verified'].value_counts()['t']/len(hior_df),4) * 100

perc_whole_apartment = (round(hior_df['room_type'].value_counts()/len(hior_df),4) * 100).iloc[0]
perc_private_room = (round(hior_df['room_type'].value_counts()/len(hior_df),4) * 100).iloc[1]
perc_shared_room = (round(hior_df['room_type'].value_counts()/len(hior_df),4) * 100).iloc[2]

accommodates = round(hior_df['accommodates'].mean(),1)

avg_price = round(hior_df['price'].mean(),2)
price_75th_percentile = round(hior_df['price'].describe()["75%"], 2)

minimum_night_75th_percentile = round(hior_df['minimum_nights'].describe()['75%'])

avg_number_of_reviews = round(hior_df['number_of_reviews'].mean())
number_of_reviews_75th_percentile = round(hior_df['number_of_reviews'].mean())

avg_security_deposit = round(hior_df['security_deposit'].mean())
avg_cleaning_fee = round(hior_df['cleaning_fee'].mean())

perc_english_description = round(((hior_df['description_lang'].value_counts()['en'] / len(hior_df['description_lang']))*100),2)

<h3> Saisonality of demand </h3>
<b>assumptions:</b>
- for the demand of a specific day we calculated a score as followed: <i>demand_dayX = 0.8 x number_of_reviews_1day_after_dayX + 0.2 x number_of_reviews_1-7days_after_dayX </i>

In [ ]:
reviews['date'] = pd.to_datetime(reviews['date'])
saisonal_df = reviews[reviews['date'] >= dt.date(year=2017, month=11, day=15)][['id', 'date']]
saisonal_df['date'] = pd.to_datetime(saisonal_df['date'])

In [ ]:
demand_df = pd.DataFrame()
demand_df['date'] = saisonal_df['date'].value_counts().sort_index().index
demand_df['reviews_per_day'] = saisonal_df['date'].value_counts().sort_index().values

In [ ]:
for i in range(len(demand_df['date'])-7):
    demand_df.at[i,'demand_score'] = 0.8*demand_df.iloc[i+1]['reviews_per_day'] + 0.2*demand_df.iloc[i+1:i+7]['reviews_per_day'].sum()
for i in range(len(demand_df['date'])-7, len(demand_df['date'])):
    demand_df.at[i,'demand_score'] = demand_df.iloc[i-1]['demand_score']

In [ ]:
demand_df['week'] = pd.Series()
week = 0
for i in range(len(demand_df['date'])):
    if(i%7 == 0):
        week = week + 1
    demand_df.at[i,'week'] = week
demand_df= demand_df[demand_df['week'] != 53]

#take data from first week to calculate demand score of last week 
temp_demand_df = demand_df[(demand_df['date'] >= dt.datetime(year=2018, month= 11, day=7)) | (demand_df['date'] <= dt.datetime(year=2017, month= 11, day=21))]
temp_demand_df['date']= temp_demand_df['date'].apply(lambda x: x.replace(year=2017))
temp_demand_df = temp_demand_df.sort_values('date')
temp_demand_df = temp_demand_df.reset_index(drop=True)

for i in range(len(temp_demand_df[temp_demand_df['week'] == 52])):
    temp_demand_df.at[i,'demand_score'] = 0.8*temp_demand_df.iloc[i+1]['reviews_per_day'] + 0.2*temp_demand_df.iloc[i+1:i+7]['reviews_per_day'].sum()

j=0
for i in range(len(demand_df['date'])-7, len(demand_df['date'])):
    demand_df.at[i,'demand_score'] = temp_demand_df.iloc[j]['demand_score']
    j=j+1

In [ ]:
plt.cla()

x = demand_df.groupby('week').sum()['demand_score'].index
y= demand_df.groupby('week').sum()['demand_score'].values

fig,ax = plt.subplots()
ax.set_xticks([0.8,5,9,13.5,18,22.5,27,31.5,36,40.5,45,49.5])
plt.subplots_adjust(bottom=0.25)
plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=90)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.plot(x,y, color='lightskyblue')

#German version 
#labels = ['November', 'Dezember','Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober']
#ax.set_xticklabels(labels, minor=False)
#plt.suptitle('Saisonalität der Nachfrage im Jahresverlauf',fontsize=12)
#plt.title('Verlauf der Nachfrage innerhalb eines Jahres', fontsize=10, color='grey', fontweight='bold')
#plt.savefig('German Graphs/saisonalität_nachfrage_jahr.png')

#English version 
labels = ['November', 'December','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October']
ax.set_xticklabels(labels, minor=False)
plt.suptitle('Saisonality of demand over the year',fontsize=12)
plt.title('Development of demand', fontsize=10, color='grey', fontweight='bold')
plt.savefig('English Graphs/saisonality_demand_year.png')

<h3> Development of demand since 2008 </h3>
- Review Rate of 50% is used to convert reviews to estimated bookings
- based on yearly changes

In [ ]:
accum_reviews_per_year= {
    "2008": 2* len(reviews[reviews['date'] <= dt.date(year=2008, month=12, day=31)]),
    "2009": 2* len(reviews[reviews['date'] <= dt.date(year=2009, month=12, day=31)]) ,
    "2010": 2* len(reviews[reviews['date'] <= dt.date(year=2010, month=12, day=31)]) ,
    "2011": 2* len(reviews[reviews['date'] <= dt.date(year=2011, month=12, day=31)]) ,
    "2012": 2* len(reviews[reviews['date'] <= dt.date(year=2012, month=12, day=31)]) ,
    "2013": 2* len(reviews[reviews['date'] <= dt.date(year=2013, month=12, day=31)]) ,
    "2014": 2* len(reviews[reviews['date'] <= dt.date(year=2014, month=12, day=31)]) ,
    "2015": 2* len(reviews[reviews['date'] <= dt.date(year=2015, month=12, day=31)]) ,
    "2016": 2* len(reviews[reviews['date'] <= dt.date(year=2016, month=12, day=31)]) ,
    "2017": 2* len(reviews[reviews['date'] <= dt.date(year=2017, month=12, day=31)]) ,
    "2018": 2* len(reviews[reviews['date'] <= dt.date(year=2018, month=12, day=31)])
}


fig = plt.figure().add_subplot(111)
x = accum_reviews_per_year.keys()
y= accum_reviews_per_year.values()

plt.xticks(fontsize=9, color='darkgrey', fontweight='bold', rotation=20)
plt.yticks(fontsize=9, color='darkgrey', fontweight='bold')
plt.subplots_adjust(left=0.15)


for i,j in zip(x,y):
    fig.annotate('%s' %j, xy=(i,j), xytext=(10,0), textcoords='offset points',)

#German version
#plt.suptitle('Wie stark ist die Nachfrageseite gewachsen?',fontsize=12)
#plt.title('Entwicklung der Nachfrage seit 2008', fontsize=10, color='grey', fontweight='bold')
#plt.ylabel('Anzahl an Buchungen')
#plt.plot(x,y, linestyle='--', marker='o', color='lightskyblue')
#plt.savefig('German Graphs/entwicklung_nachfrage.png')

#English version
plt.suptitle('How strong was the growth on the demand side?',fontsize=12)
plt.title('Development of demand since 2008', fontsize=10, color='grey', fontweight='bold')
plt.ylabel('Bookings')
plt.plot(x,y, linestyle='--', marker='o', color='lightskyblue')
plt.savefig('English Graphs/development_demand.png')
